In [1]:
import hopsworks
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os

def train_model():
    print("Logging into Hopsworks...")
    project = hopsworks.login()
    fs = project.get_feature_store()

    # Define Feature View name and version
    fv_name = "azores_wind_fv"
    fv_version = 1
    
    # 1. Get Feature Group
    print("Creating or retrieving Feature View...")
    wind_fg = fs.get_feature_group(name="azores_wind_data", version=1)
    
    # 2. Select all features for the query
    query = wind_fg.select_all()
    
    # Create Feature View (Handle existence check in production, simplified here)
    feature_view = fs.create_feature_view(
        name=fv_name,
        version=fv_version,
        description="Predict wind speed based on forecast",
        labels=["wind_speed"], # The target variable we want to predict
        query=query
    )
    
    print(f"Feature View ready.")
    
    # 3. Split into Train and Test sets
    print("Splitting data into Train/Test sets...")
    X_train, X_test, y_train, y_test = feature_view.train_test_split(
        test_size=0.2,
        description="Flores wind dataset split"
    )

    # Data Cleanup: Remove non-feature columns and target if present in X
    drop_cols = ["date", "date_str", "wind_speed"]
    # Ensure only feature columns remain: temp, precip, gusts, direction
    X_train = X_train.drop(columns=[c for c in drop_cols if c in X_train.columns])
    X_test = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])
    
    print(f"Training features: {X_train.columns.tolist()}")

    # 4. Train Model (Using Histogram Gradient Boosting)
    print("Training HistGradientBoostingRegressor...")
    model = HistGradientBoostingRegressor(
        max_iter=100,
        learning_rate=0.1, 
        max_depth=5,
        random_state=42
    )
    model.fit(X_train, y_train)

    # 5. Evaluate Model
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    r2 = r2_score(y_test, preds)
    
    print(f"  Model Evaluation Results:")
    print(f"   RMSE: {rmse:.2f} km/h")
    print(f"   R2 Score: {r2:.2f}")

    # 6. Save and Register Model
    print("Saving model to Hopsworks Model Registry...")
    
    model_dir = "boat_model"
    if not os.path.isdir(model_dir):
        os.mkdir(model_dir)
    joblib.dump(model, model_dir + "/azores_wind_model.pkl")

    mr = project.get_model_registry()
    
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    
    input_schema = Schema(X_train)
    output_schema = Schema(y_train)
    model_schema = ModelSchema(input_schema, output_schema)

    azores_model = mr.python.create_model(
        name="azores_wind_model", 
        metrics={"rmse": rmse, "r2": r2},
        model_schema=model_schema,
        input_example=X_train.sample(), 
        description="Predicts daily max wind speed for Flores Island"
    )
    
    azores_model.save(model_dir)
    print("Model successfully registered!")

if __name__ == "__main__":
    train_model()

🔐 Logging into Hopsworks...
2026-01-13 22:27:30,874 INFO: Initializing external client
2026-01-13 22:27:30,874 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-13 22:27:31,513 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-13 22:27:32,480 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1303706
🆕 Creating or retrieving Feature View...
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1303706/fs/1291336/fv/azores_wind_fv/version/1
✅ Feature View ready.
✂️ Splitting data into Train/Test sets...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.09s) 
2026-01-13 22:27:43,557 WARNING: VersionWarning: Incremented version to `1`.

Training features: ['temperature_max', 'precipitation', 'wind_gusts', 'wind_direction']
🧠 Training HistGradientBoostingRegressor...
2026-01-13 22:27:43,559 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-13 22:27:43,560 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` in

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/wenbo/Desktop/project/boat_model/azores_wind_model.pkl: 0.000%|          | 0/203984 elapsed<0…

Uploading /Users/wenbo/Desktop/project/input_example.json: 0.000%|          | 0/80 elapsed<00:00 remaining<?

Uploading /Users/wenbo/Desktop/project/model_schema.json: 0.000%|          | 0/490 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1303706/models/azores_wind_model/1
🎉 Model successfully registered!
